In [20]:
from faker import Faker
from faker.providers import BaseProvider
import random
import csv

# create ParticipatsProvider class 
class ParticipantsProvider(BaseProvider):
    def participants(self, num_participants=100):
        fake = Faker()
        participants = []
        for _ in range(num_participants):
            participant = {
                'participant_id': random.randint(1, 999),
                'first_name': fake.name(),
                'last_name': fake.last_name(),
                'department': fake.job(),
                'region': fake.state(),
                'start_date_at_company': fake.date_between(start_date='-5y', end_date='today')
            }
            participants.append(participant)
        return participants

# EngagementProvider
class EngagementProvider(BaseProvider):
    def engagement(self, num_engagements=10):
        fake = Faker()
        engagements = []
        for _ in range(num_engagements):
            engagement = {
                'engagement_id': random.randint(1, 999),
                'participant_id': random.randint(1, 999),
                'activity_type': fake.sentence(),
                'timestamp': fake.date_time_this_year(),
                'duration_minutes': random.randint(1, 120),
            }
            engagements.append(engagement)
        return engagements
    
# EnrollmentProvider
class EnrollmentProvider(BaseProvider):
    def enrollments(self, num_enrollments=99):
        fake = Faker()
        enrollments = []
        for _ in range(num_enrollments):
            enrollment = {
                'enrollment_id': random.randint(1, 999),
                'participant_id': random.randint(1, 999),
                'course_id': random.randint(1, 999),
                'enrollment_date': fake.date_between(start_date='-2y', end_date='today'),
                'completion_status': random.choice(['completed', 'in_progress', 'not_started']),
                'score': random.randint(0, 100) if random.choice([True, False]) else None,
            }
            enrollments.append(enrollment)
        return enrollments

# CourseProvider
class CourseProvider(BaseProvider):
    def courses(self, num_courses=50):
        fake = Faker()
        courses = []
        for _ in range(num_courses):
            course = {
                'course_id': random.randint(1, 999),
                'course_name': fake.catch_phrase(),
                'instructor': fake.name(),
                'duration_hours': random.randint(1, 40),
                'category': random.choice(['Technology', 'Business', 'Health', 'Arts']),
            }
            courses.append(course)
        return courses

# FeedbackProvider
class FeedbackProvider(BaseProvider):
    def feedback(self, num_feedbacks=100):
        fake = Faker()
        feedbacks = []
        for _ in range(num_feedbacks):
            feedback = {
                'feedback_id': random.randint(1, 999),
                'enrollment_id': random.randint(1, 999),
                'rating': random.randint(1, 5),
                'comments': fake.text(max_nb_chars=200),
                'submitted_at': fake.date_time_this_year(),
            }
            feedbacks.append(feedback)
        return feedbacks
    
fake = Faker()
fake.add_provider(ParticipantsProvider)
fake.add_provider(EngagementProvider)
fake.add_provider(EnrollmentProvider)
fake.add_provider(CourseProvider)
fake.add_provider(FeedbackProvider)

def generate_data():
    participants = fake.participants(100)
    engagements = fake.engagement(10)
    enrollments = fake.enrollments(99)
    courses = fake.courses(50)
    feedbacks = fake.feedback(100)

    return {
        'participants': participants,
        'engagements': engagements,
        'enrollments': enrollments,
        'courses': courses,
        'feedbacks': feedbacks
    }
def save_to_csv(data, filename):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=data[0].keys())
        writer.writeheader()
        for row in data:
            writer.writerow(row)
def main():
    data = generate_data()
    
    save_to_csv(data['participants'], 'participants.csv')
    save_to_csv(data['engagements'], 'engagements.csv')
    save_to_csv(data['enrollments'], 'enrollments.csv')
    save_to_csv(data['courses'], 'courses.csv')
    save_to_csv(data['feedbacks'], 'feedbacks.csv')
    
if __name__ == "__main__":
    main()
    print("Data generation complete. CSV files created.")



Data generation complete. CSV files created.
